In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
import os

In [ ]:
%load_ext tensorboard

In [ ]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [ ]:
mnist_train, mnist_test = datasets['train'], datasets['test']

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
print('Num of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

In [ ]:
print(num_train_examples)
print(num_test_examples)

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [ ]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

In [ ]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BUFFER_SIZE)

In [ ]:
test_dataset = mnist_test.map(scale).batch(BUFFER_SIZE)

In [ ]:
with strategy.scope():
  model = tf.keras.Sequential([
                               tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
                               tf.keras.layers.MaxPooling2D(),
                               tf.keras.layers.Flatten(),
                               tf.keras.layers.Dense(64, activation='relu'),
                               tf.keras.layers.Dense(10)
  ])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
# Define the checkpoint directory to store the checkpoints.
checkpoint_dir = './training_checkpoints'
# Define the name of the checkpoint files.
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [ ]:
# Define a function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [ ]:
# Define a callback for printing the learning rate at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))

In [ ]:
# Put all the callbacks together.
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [ ]:
EPOCHS=12
model.fit(train_dataset, epochs=EPOCHS, callbacks=callbacks)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
eval_loss, eval_acc = model.evaluate(test_dataset)

In [ ]:
print(eval_loss, eval_acc)

In [ ]:
%tensorboard --logdir=logs